In [1]:
import openml
import pandas as pd
import warnings

from pymfe.mfe import MFE

import warnings
warnings.filterwarnings("ignore")

d:\repos\AutoMLLabs\lab2\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Получение данные с openml

In [2]:
ALL_DATASETS = openml.datasets.list_datasets(output_format="dataframe")

In [3]:
ALL_DATASETS.dropna(axis=0, inplace=True)
ALL_DATASETS

,did,name,version,uploader,status,format,MajorityClassSize,MaxNominalAttDistinctValues,MinorityClassSize,NumberOfClasses,NumberOfFeatures,NumberOfInstances,NumberOfInstancesWithMissingValues,NumberOfMissingValues,NumberOfNumericFeatures,NumberOfSymbolicFeatures
2,2,anneal,1,1,active,ARFF,684.0,7.0,8.0,5.0,39.0,898.0,898.0,22175.0,6.0,33.0
3,3,kr-vs-kp,1,1,active,ARFF,1669.0,3.0,1527.0,2.0,37.0,3196.0,0.0,0.0,0.0,37.0
4,4,labor,1,1,active,ARFF,37.0,3.0,20.0,2.0,17.0,57.0,56.0,326.0,8.0,9.0
5,5,arrhythmia,1,1,active,ARFF,245.0,13.0,2.0,13.0,280.0,452.0,384.0,408.0,206.0,74.0
6,6,letter,1,1,active,ARFF,813.0,26.0,734.0,26.0,17.0,20000.0,0.0,0.0,16.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42172,42172,regime_alimentaire,1,4030,active,ARFF,161.0,7.0,41.0,2.0,20.0,202.0,17.0,17.0,3.0,17.0
42186,42186,JuanFeldmanIris,1,10443,active,ARFF,50.0,3.0,50.0,3.0,5.0,150.0,0.0,0.0,4.0,1.0
42192,42192,compas-two-years,3,10483,active,ARFF,2795.0,2.0,2483.0,2.0,14.0,5278.0,0.0,0.0,7.0,7.0
42193,42193,compas-two-years,4,10483,active,ARFF,2795.0,2.0,2483.0,2.0,14.0,5278.0,0.0,0.0,7.0,7.0


# 2. Выбор метапризнаков


In [4]:
feature_dict = []

In [5]:
def get_MFE(df, name):
    features = df.drop("class", axis=1)
    target = df["class"]  
    mfe = MFE(groups=["general", "statistical", "model-based"])
    mfe.fit(features.values, target.to_list())
    ft = mfe.extract()
    df_dict = dict(zip(*ft))
    df_dict["name"] = name
    
    return df_dict

In [6]:
def mfe_generator(datasets: pd.DataFrame):
        for ds in datasets.to_dict(orient="records"):
            try:
                df = openml.datasets.get_dataset(ds["name"])
                df = df.get_data(dataset_format='datarame')[0]
   
                
                yield get_MFE(df, ds['name'])   
            except:
                continue

pd.DataFrame.from_dict(data=[d for d in mfe_generator(ALL_DATASETS[:15])], orient='columns')

,attr_to_inst,can_cor.mean,can_cor.sd,cat_to_num,cor.mean,cor.sd,cov.mean,cov.sd,eigenvalues.mean,eigenvalues.sd,...,tree_imbalance.mean,tree_imbalance.sd,tree_shape.mean,tree_shape.sd,var.mean,var.sd,var_importance.mean,var_importance.sd,w_lambda,name
0,0.011264,0.799823,NaN,NaN,0.085644,0.108167,0.011234,0.018813,1.303675e-01,1.595867e-01,...,0.054833,0.092559,0.052246,0.093870,1.303675e-01,8.510044e-02,0.026316,0.063276,3.602836e-01,kr-vs-kp
1,0.617257,NaN,NaN,0.354369,NaN,NaN,NaN,NaN,NaN,NaN,...,0.042300,0.081667,0.045858,0.082127,NaN,NaN,NaN,NaN,NaN,arrhythmia
2,0.000800,0.467835,0.277919,0.000000,0.182896,0.208834,0.949368,1.150810,5.344024e+00,6.297241e+00,...,0.038324,0.053138,0.003273,0.013876,5.344024e+00,2.054562e+00,0.062500,0.041448,9.799904e-04,letter
3,0.121622,0.924706,0.080230,5.000000,0.129825,0.124294,0.028216,0.054487,2.763821e-01,7.087613e-01,...,0.164807,0.108745,0.171224,0.101704,2.763821e-01,6.150475e-01,0.030303,0.056543,1.671336e-07,lymph
4,0.006400,0.423927,0.563810,0.000000,0.000000,0.000000,0.000000,0.000000,2.003205e+00,0.000000e+00,...,0.125015,0.074884,0.040688,0.038232,2.003205e+00,0.000000e+00,0.250000,0.021671,3.231216e-01,balance-scale
5,0.108000,0.927064,0.048168,0.000000,0.249626,0.196869,724.186099,1725.167867,4.581543e+03,2.805878e+04,...,0.057948,0.084058,0.040307,0.073197,4.581543e+03,5.584125e+03,0.004630,0.015210,3.206439e-09,mfeat-factors
6,0.038000,0.721437,0.230074,0.000000,0.145891,0.103520,0.000717,0.000983,5.520283e-03,1.212796e-02,...,0.051776,0.075193,0.023346,0.049591,5.520283e-03,5.878874e-03,0.013158,0.027362,5.040808e-05,mfeat-fourier
7,0.032000,0.809847,0.114468,0.000000,0.107200,0.083368,0.552460,0.924340,6.491589e+00,1.301381e+01,...,0.079028,0.087293,0.032919,0.045479,6.491589e+00,9.755558e+00,0.015625,0.029717,1.256370e-05,mfeat-karhunen
8,0.003000,0.601871,0.343177,0.000000,0.572757,0.321310,6498.553950,23504.324455,2.353407e+06,5.764630e+06,...,0.026064,0.049207,0.009814,0.036120,2.353407e+06,5.764340e+06,0.166667,0.075151,1.517596e-03,mfeat-morphological


### Выбираем набор данных

In [7]:
ALL_DATASETS[ALL_DATASETS["NumberOfInstances"] <= 1000]

,did,name,version,uploader,status,format,MajorityClassSize,MaxNominalAttDistinctValues,MinorityClassSize,NumberOfClasses,NumberOfFeatures,NumberOfInstances,NumberOfInstancesWithMissingValues,NumberOfMissingValues,NumberOfNumericFeatures,NumberOfSymbolicFeatures
2,2,anneal,1,1,active,ARFF,684.0,7.0,8.0,5.0,39.0,898.0,898.0,22175.0,6.0,33.0
4,4,labor,1,1,active,ARFF,37.0,3.0,20.0,2.0,17.0,57.0,56.0,326.0,8.0,9.0
5,5,arrhythmia,1,1,active,ARFF,245.0,13.0,2.0,13.0,280.0,452.0,384.0,408.0,206.0,74.0
7,7,audiology,1,1,active,ARFF,57.0,24.0,1.0,24.0,70.0,226.0,222.0,317.0,0.0,70.0
9,9,autos,1,1,active,ARFF,67.0,22.0,3.0,6.0,26.0,205.0,46.0,59.0,15.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42098,42098,iris,45,7214,active,ARFF,50.0,3.0,50.0,3.0,5.0,150.0,0.0,0.0,4.0,1.0
42167,42167,stress,1,4030,active,ARFF,159.0,5.0,3.0,3.0,13.0,202.0,172.0,202.0,8.0,5.0
42169,42169,epiparo_extract,1,4030,active,ARFF,123.0,5.0,1.0,6.0,20.0,224.0,143.0,205.0,9.0,11.0
42172,42172,regime_alimentaire,1,4030,active,ARFF,161.0,7.0,41.0,2.0,20.0,202.0,17.0,17.0,3.0,17.0


In [8]:
ALL_DATASETS[ALL_DATASETS["NumberOfInstances"] <= 5000]

,did,name,version,uploader,status,format,MajorityClassSize,MaxNominalAttDistinctValues,MinorityClassSize,NumberOfClasses,NumberOfFeatures,NumberOfInstances,NumberOfInstancesWithMissingValues,NumberOfMissingValues,NumberOfNumericFeatures,NumberOfSymbolicFeatures
2,2,anneal,1,1,active,ARFF,684.0,7.0,8.0,5.0,39.0,898.0,898.0,22175.0,6.0,33.0
3,3,kr-vs-kp,1,1,active,ARFF,1669.0,3.0,1527.0,2.0,37.0,3196.0,0.0,0.0,0.0,37.0
4,4,labor,1,1,active,ARFF,37.0,3.0,20.0,2.0,17.0,57.0,56.0,326.0,8.0,9.0
5,5,arrhythmia,1,1,active,ARFF,245.0,13.0,2.0,13.0,280.0,452.0,384.0,408.0,206.0,74.0
7,7,audiology,1,1,active,ARFF,57.0,24.0,1.0,24.0,70.0,226.0,222.0,317.0,0.0,70.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42123,42123,article_influence,2,3422,active,arff,32.0,3169.0,1.0,3169.0,7.0,3615.0,12.0,48.0,5.0,1.0
42167,42167,stress,1,4030,active,ARFF,159.0,5.0,3.0,3.0,13.0,202.0,172.0,202.0,8.0,5.0
42169,42169,epiparo_extract,1,4030,active,ARFF,123.0,5.0,1.0,6.0,20.0,224.0,143.0,205.0,9.0,11.0
42172,42172,regime_alimentaire,1,4030,active,ARFF,161.0,7.0,41.0,2.0,20.0,202.0,17.0,17.0,3.0,17.0


In [9]:
import pickle

res = []
for ds in ALL_DATASETS[ALL_DATASETS["NumberOfInstances"] <= 5000].to_dict("records"):
    try:
        df = openml.datasets.get_dataset(ds["name"])
        df = df.get_data(dataset_format="dataframe")[0]
        if 'class' in df.columns:
            res.append((ds["name"], df))
    except:
        continue
    

with open("datasets.pk", "wb") as f:
    pickle.dump(res, f)



In [10]:
with open("datasets.pk", "rb") as f:
    res = pickle.load(f)

len(res)

186

In [11]:
selected_dataset = ALL_DATASETS[ALL_DATASETS["did"]==24]
selected_dataset

,did,name,version,uploader,status,format,MajorityClassSize,MaxNominalAttDistinctValues,MinorityClassSize,NumberOfClasses,NumberOfFeatures,NumberOfInstances,NumberOfInstancesWithMissingValues,NumberOfMissingValues,NumberOfNumericFeatures,NumberOfSymbolicFeatures
24,24,mushroom,1,1,active,ARFF,4208.0,12.0,3916.0,2.0,23.0,8124.0,2480.0,2480.0,0.0,23.0


In [12]:
df_d = selected_dataset.to_dict("records")
name = df_d[0]["name"]

selected_df = openml.datasets.get_dataset(name)
selected_df = selected_df.get_data(dataset_format="dataframe")[0]
selected_df.dropna(inplace=True, axis=0)
selected_df

,cap-shape,cap-surface,cap-color,bruises%3F,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,class
0,x,s,n,t,p,f,c,n,k,e,...,w,w,p,w,o,p,k,s,u,p
1,x,s,y,t,a,f,c,b,k,e,...,w,w,p,w,o,p,n,n,g,e
2,b,s,w,t,l,f,c,b,n,e,...,w,w,p,w,o,p,n,n,m,e
3,x,y,w,t,p,f,c,n,n,e,...,w,w,p,w,o,p,k,s,u,p
4,x,s,g,f,n,f,w,b,k,t,...,w,w,p,w,o,e,n,a,g,e
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7986,b,y,n,f,n,f,c,b,w,e,...,n,n,p,w,t,p,w,y,p,e
8001,x,y,n,f,n,f,c,b,w,e,...,n,n,p,w,t,p,w,y,p,e
8038,x,y,g,t,n,f,c,b,w,e,...,w,w,p,w,t,p,w,y,p,e
8095,x,y,c,f,m,f,c,b,y,e,...,c,c,p,w,n,n,w,c,d,p


In [13]:
mfe_sel_df = get_MFE(selected_df, name="selected")
mfe_sel_df

{'attr_to_inst': 0.0038979447200566974,
 'can_cor.mean': 1.0,
 'can_cor.sd': nan,
 'cat_to_num': nan,
 'cor.mean': 0.11779457927924072,
 'cor.sd': 0.14769199628012536,
 'cov.mean': 0.012899556765958125,
 'cov.sd': 0.0239765578128356,
 'eigenvalues.mean': 0.10612419408312239,
 'eigenvalues.sd': 0.25508835330834534,
 'freq_class.mean': 0.5,
 'freq_class.sd': 0.16687920491503921,
 'g_mean.mean': 0.0,
 'g_mean.sd': 0.0,
 'gravity': 2.1267135533271366,
 'h_mean.mean': 0.0,
 'h_mean.sd': 0.0,
 'inst_to_attr': 256.54545454545456,
 'iq_range.mean': 0.23376623376623376,
 'iq_range.sd': 0.4260004909838997,
 'kurtosis.mean': 100.0244219117455,
 'kurtosis.sd': 259.5788385503079,
 'leaves': 9,
 'leaves_branch.mean': 3.7777777777777777,
 'leaves_branch.sd': 1.2018504251546631,
 'leaves_corrob.mean': 0.1111111111111111,
 'leaves_corrob.sd': 0.19260744248219652,
 'leaves_homo.mean': 37.46666666666667,
 'leaves_homo.sd': 13.142298124757328,
 'leaves_per_class.mean': 0.5,
 'leaves_per_class.sd': 0.07856

In [14]:
sample_sel_df = selected_df.sample(len(selected_df), axis=0)
get_MFE(sample_sel_df, "sample_rows")

{'attr_to_inst': 0.0038979447200566974,
 'can_cor.mean': 0.9999999999999996,
 'can_cor.sd': nan,
 'cat_to_num': nan,
 'cor.mean': 0.11779457927924077,
 'cor.sd': 0.14769199628012544,
 'cov.mean': 0.012899556765958125,
 'cov.sd': 0.023976557812835596,
 'eigenvalues.mean': 0.10612419408312239,
 'eigenvalues.sd': 0.2550883533083454,
 'freq_class.mean': 0.5,
 'freq_class.sd': 0.16687920491503921,
 'g_mean.mean': 0.0,
 'g_mean.sd': 0.0,
 'gravity': 2.1267135533271366,
 'h_mean.mean': 0.0,
 'h_mean.sd': 0.0,
 'inst_to_attr': 256.54545454545456,
 'iq_range.mean': 0.23376623376623376,
 'iq_range.sd': 0.4260004909838997,
 'kurtosis.mean': 100.02442191174553,
 'kurtosis.sd': 259.578838550308,
 'leaves': 9,
 'leaves_branch.mean': 3.7777777777777777,
 'leaves_branch.sd': 1.2018504251546631,
 'leaves_corrob.mean': 0.1111111111111111,
 'leaves_corrob.sd': 0.19260744248219652,
 'leaves_homo.mean': 37.46666666666667,
 'leaves_homo.sd': 13.142298124757328,
 'leaves_per_class.mean': 0.5,
 'leaves_per_cl

In [15]:
sample_sel_df = sample_sel_df.sample(len(sample_sel_df.columns), axis=1)
get_MFE(sample_sel_df, "sample_columns")

{'attr_to_inst': 0.0038979447200566974,
 'can_cor.mean': 1.0,
 'can_cor.sd': nan,
 'cat_to_num': nan,
 'cor.mean': 0.11760382242424426,
 'cor.sd': 0.14780786006293176,
 'cov.mean': 0.012941864537709753,
 'cov.sd': 0.02402368295835662,
 'eigenvalues.mean': 0.10629408618211561,
 'eigenvalues.sd': 0.2555609223285139,
 'freq_class.mean': 0.5,
 'freq_class.sd': 0.16687920491503921,
 'g_mean.mean': 0.0,
 'g_mean.sd': 0.0,
 'gravity': 2.128825034590379,
 'h_mean.mean': 0.0,
 'h_mean.sd': 0.0,
 'inst_to_attr': 256.54545454545456,
 'iq_range.mean': 0.23376623376623376,
 'iq_range.sd': 0.4260004909838997,
 'kurtosis.mean': 99.97121284212336,
 'kurtosis.sd': 259.5971516743465,
 'leaves': 9,
 'leaves_branch.mean': 3.7777777777777777,
 'leaves_branch.sd': 1.2018504251546631,
 'leaves_corrob.mean': 0.1111111111111111,
 'leaves_corrob.sd': 0.19260744248219652,
 'leaves_homo.mean': 37.46666666666667,
 'leaves_homo.sd': 13.142298124757328,
 'leaves_per_class.mean': 0.5,
 'leaves_per_class.sd': 0.078567